# Colombian Constitution EDA.
    Brief resume about the methodology and the results that we got.

## About the Document
    The Political Constitution of the Republic of Colombia of 1991 is the Magna Carta of the Republic of Colombia. It was promulgated in the Constitutional Gazette number 114 of Thursday, July 4, 1991, and is also known as the Constitution of Human Rights. It replaced the Political Constitution of 1886 and was issued during the presidency of the liberal César Gaviria.1

## DS Pipeline



## Document Data Model

PowerPoint graph, meanwhile we can use the hierarchy of the articles dictionary

```
hierarchy = {
    'TITULO' : 'h1',
    'DISPOSICIONES' : 'h1',
    'CAPITULO' : 'h2',
    'ARTÍCULO' : 'p'

}
```

## EDA

### Libraries 

In [1]:
import numpy as np
import pandas as pd
# For visualizations
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub
import tensorflow_text
from os import path




Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
root_folder = "LegalSearcher/ElasticSearch"
constitution_f = f'{root_folder}/embedding.json'
filepath = path.abspath(path.join('','..', '..', constitution_f))
#import Dataset
df = pd.read_json(filepath)

In [3]:
print("Downloading pre-trained embeddings from tensorflow hub...")
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

text_ph = tf.placeholder(tf.string)
embeddings = embed(text_ph)
print("Done.")

print("Creating tensorflow session...")
session = tf.Session()
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
print("Done.")

Done.
Creating tensorflow session...
Done.


In [4]:
##### EMBEDDING #####

def embed_text(text):
    vectors = session.run(embeddings, feed_dict={text_ph: text})
    return [vector.tolist() for vector in vectors]

In [5]:
df=df[['id','embedding']]
df.replace(r'\n','', regex=True,inplace=True) 
df['embedding'] = df['embedding'].apply(lambda x:x[1:])
df.rename(columns={'embedding':'article_list'},inplace=True)
df.head()


,id,article_list
0,const01000001,"[Colombia es un Estado social de derecho, orga..."
1,const01000002,[Son fines esenciales del Estado: servir a la ...
2,const01000003,[La soberanía reside exclusivamente en el pueb...
3,const01000004,"[La Constitución es norma de normas., En todo ..."
4,const01000005,"[El Estado reconoce, sin discriminación alguna..."


In [6]:
df['embedds'] = df['article_list'].apply(lambda x:embed_text(x)[0])
df.head()
# art1_res = embed(art1)

,id,article_list,embedds
0,const01000001,"[Colombia es un Estado social de derecho, orga...","[0.024026719853281975, -0.0791270062327385, 0...."
1,const01000002,[Son fines esenciales del Estado: servir a la ...,"[0.058738261461257935, 0.07823820412158966, -0..."
2,const01000003,[La soberanía reside exclusivamente en el pueb...,"[-0.003973111975938082, -0.007816542871296406,..."
3,const01000004,"[La Constitución es norma de normas., En todo ...","[0.06180394068360329, -0.02205243706703186, 0...."
4,const01000005,"[El Estado reconoce, sin discriminación alguna...","[0.027791934087872505, -0.018517600372433662, ..."


In [22]:
query0 = embed_text(['Colombia es un Estado social de derecho, organizado en forma de República unitaria, descentralizada, con autonomía de sus entidades territoriales, democrática, participativa y pluralista, fundada en el respeto de la dignidad humana, en el trabajo y la solidaridad de las personas que la integran y en la prevalencia del interés general'])[0]
query1 = embed_text(['organizacion de colombia'])[0]
query2 = embed_text(['Fines esenciales del estado'])[0]
query3 = embed_text(['Representantes del pueblo'])[0]
query4 = embed_text(['derechos humanos'])[0]
query5 = embed_text(['tratados y convenios'])[0]
query6 = embed_text(['Lorem ipsum dolor sit amet'])[0]
query7 = embed_text(['organos del poder publico'])[0]
query8 = embed_text(['Propiedad Intelectual'])[0]
query9 = embed_text(['cuidado del medio ambiente'])[0]
query10 = embed_text(['Ley de 1992'])[0]
query11 = embed_text(['pizza'])[0]

In [23]:
# session.close

In [24]:
# Cosine Similarity Function
def similarity(A,B):
    cos_sim = np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))+1
    return cos_sim

In [27]:
# A = Article in Constitution
B = query11
count = 0
res = []
for index, row in df.iterrows():
    cos_sim = similarity(row['embedds'],B)
    if  cos_sim > count:
        count = cos_sim
        res = [index, row['article_list'],count]
    elif count < 1.2:
        res = "Sorry parce we Couldn't find anything about that in the Constitution."
    
# print(f'Query ingresada: {B}')    
print(res)

Sorry parce we Couldn't find anything about that in the Constitution.


In [7]:
from fastparquet import write 
write('Cons.parq', df)